In [2]:
import os
import re
import sys

In [3]:
cyrest_folder="/Users/jboucas/py2cytoscape/py2cytoscape/cyrest/"
docs_folder="/Users/jboucas/Desktop/tmp_docs_py2cy/"

In [4]:
files=os.listdir(cyrest_folder)
no_files=["cyrest.py","__init__.py",'base.py']
files=[ s for s in files if s not in no_files ]
files=[ s for s in files if "pyc" not in s ]
files=[ s for s in files if s in ["table.py", ]]

In [18]:
for f in ["cyndex2.py","apply.py","layout.py","styles.py","table.py","session.py","enrichmentmap.py","apps.py","ui.py","collections.py","networks.py"]:
    #for f in ["table.py"]:
    print f
    sys.stdout.flush()
    functions={}
    mainspace=f.split(".")[0]
    print mainspace
    with open(cyrest_folder+f, "r") as fin:
        funcstring=None
        for l in fin:
            l_=l.rstrip("\n")
            if ("    def " in l_) & ("__" not in l_):
                fname=re.search('def (.*)\(', l_)
                fname=fname.group(1)
                func_title="## ***cyclient.%s.%s***" %(mainspace,fname)
                #print l_
                if "\\" not in l_:
                    func=re.search('def (.*):', l_)
                    func=func.group(1)
                    func="**`cyclient.%s.%s`**" %(mainspace,func)
                    functions[fname]={'title':func_title,'func':func}
                else:
                    funcstring=l_.strip("\\")
                    
            elif funcstring:
                funcstring=funcstring+l_
                if ":" in funcstring:
                    func=re.search('def (.*):', funcstring)
                    func=func.group(1)
                    
                    funcstring=None    
                    func="**`cyclient.%s.%s`**" %(mainspace,func)
                    functions[fname]={'title':func_title,'func':func}
        
    
    with open(cyrest_folder+f, "r") as fin:
        collect=False
        doc=[]
        function_found=False
        for l in fin:
            l_=l.rstrip("\n")
            if ("    def " in l_) & ("__" not in l_):
                fname=re.search('def (.*)\(', l_)
                fname=fname.group(1)
                func_title="## ***cyclient.%s.%s***" %(mainspace,fname) 
                function_found=True
            if ('"""' in l_ ) & (len( [s for s in doc if '"""' in s ]) == 0 ) & (function_found):
                collect=True
            if ('"""' in l_ ) & (len([s for s in doc if '"""' in s ]) == 1  ) & (function_found):
                functions[fname]["docs"]=doc
                doc=[]
                collect=False
            if (collect) & (function_found):
                doc.append( l_.rstrip("\n") )
    
    with open(docs_folder+f.replace(".py", ".md"), "w") as fout:
                
        valid_functions=functions.keys()
        valid_functions=[ s for s in valid_functions if s != "target"]
        
        for docf in valid_functions:

            try:
                doc_=functions[docf]["docs"][1:]
            except:
                print f, docf, functions[docf]
                doc_=functions[docf]["docs"][1:]
            
            collect=True
            description=[]
            for s in doc_:
                if (":param" not in s) & (collect==True):
                    desc=s.strip(" ")
                    if len(desc) > 0:
                        description.append(desc)
                elif ":param" in s:
                    collect=False
            
            description="\n".join(description)

            params=[]
            params_={}
            pname=None
            for p in doc_:
                if ":param " in p:
                    #print p
                    pname=re.search(':param (.*):', p)
                    pname=pname.group(1)
                    params.append(pname)
                    pdesc=p.split("%s:"%pname)[-1].strip(" ")
                    params_[pname]=pdesc
                elif pname:
                    pdesc=params_[pname]
                    pdesc=pdesc+"\n"+p.strip(" ")
                    params_[pname]=pdesc
            params_text=[]
            for p in params:
                text="* **`%s`** %s" %(p, params_[p])
                params_text.append(text)


            ftitle=functions[docf]["title"]
            func=functions[docf]["func"]
            docs_text=ftitle+"\n\n"+func+"\n\n"+description+"\n\n"+"\n".join(params_text)+"\n\n___\n\n"
            docs_text=docs_text.replace(":returns:", "* **`returns`**")
            
            print docs_text
        
            fout.write(docs_text)

cyndex2.py
cyndex2
## ***cyclient.cyndex2.saveNetworkToNdex***

**`cyclient.cyndex2.saveNetworkToNdex(suid, body, verbose=None)`**

Save a network/collection to NDEx

* **`suid`** Cytoscape Collection/Subnetwork SUID
* **`body`** Properties required to save network to NDEx.
* **`verbose`** print more

* **`returns`** 200: successful operation; 404: Network does not exist

___


## ***cyclient.cyndex2.getAppInfo***

**`cyclient.cyndex2.getAppInfo(verbose=None)`**

App version and other basic information will be provided.

* **`verbose`** print more

* **`returns`** 200: successful operation

___


## ***cyclient.cyndex2.updateCurrentNetworkInNdex***

**`cyclient.cyndex2.updateCurrentNetworkInNdex(body, verbose=None)`**

Update current network's record in NDEx

* **`body`** Properties required to update a network record in NDEx.
* **`verbose`** print more

* **`returns`** 200: successful operation; 404: Network does not exist

___


## ***cyclient.cyndex2.getNetworkSummary***

**`cyclien

table
## ***cyclient.table.import_url***

**`cyclient.table.import_url(self,caseSensitiveNetworkCollectionKeys=None,        caseSensitiveNetworkKeys=None,dataTypeList=None,\        DataTypeTargetForNetworkCollection=None,DataTypeTargetForNetworkList=None,\        delimiters=None,delimitersForDataList=None,firstRowAsColumnNames=None,\        KeyColumnForMapping=None,KeyColumnForMappingNetworkList=None,\        keyColumnIndex=None,newTableName=None,startLoadRow=None,\        TargetNetworkCollection=None,TargetNetworkList=None,url=None,\        WhereImportTable=None,verbose=None)`**

Similar to Import Table this uses a long list of input parameters to
specify the attributes of the table, the mapping keys, and the destination
table for the input.

* **`caseSensitiveNetworkCollectionKeys (string, optional)`** Determines wh
ether capitalization is considered in matching and sorting
* **`caseSensitiveNetworkKeys (string, optional)`** Determines whether capi
talization is considered in matchin

session
## ***cyclient.session.save_as***

**`cyclient.session.save_as(self, session_file, verbose=False)`**

Saves the current session as a new file.

* **`session_file`** The path to the file where the current session
must be saved to.
* **`verbose`** print more

___


## ***cyclient.session.runGarbageCollection***

**`cyclient.session.runGarbageCollection(verbose=None)`**

Manually call Java's System.gc() to free up unused memory. This process happens automatically, but may be useful to call explicitly for testing or evaluation purposes.

* **`verbose`** print more

* **`returns`** 204: Successful Garbage Collection

___


## ***cyclient.session.getSessionName***

**`cyclient.session.getSessionName(verbose=None)`**

Returns the file name for the current Cytoscape session.

* **`verbose`** print more

* **`returns`** 200: successful operation

___


## ***cyclient.session.getSessionFromFile***

**`cyclient.session.getSessionFromFile(file, verbose=None)`**

Loads a session from a loca